In [2]:
import sys
import numpy as np
import pandas as pd
#from scipy.misc import imread
from matplotlib.pyplot import imread
import pickle
import os
import matplotlib.pyplot as plt
#%matplotlib inline

#import cv2
import time

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
import tensorflow.keras.initializers as initializers
from tensorflow.keras.models import Model

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform

from tensorflow.keras.layers import Layer
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K

from sklearn.utils import shuffle

import numpy.random as rng

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
from os import walk
import torch

import matplotlib.pyplot as plt
import os

import torchaudio
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset,Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
import random
#from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.utils.data as data_utils
import os

In [5]:
import torch
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
import matplotlib.pyplot as plt

In [55]:
train_folder = "D:/COVID_COUGH_SOUNDS/50S_overlap_458/50msOverlap_chunks_notAUG/train/"
val_folder = 'D:/COVID_COUGH_SOUNDS/50S_overlap_458/50msOverlap_chunks_notAUG/test/'
save_path = 'D:/COVID_COUGH_SOUNDS/1second_chunks_458/balanced_1s_not_aug/one_shot1sec/save_path'
model_path = 'D:/COVID_COUGH_SOUNDS/1second_chunks_458/balanced_1s_not_aug/one_shot1sec/save_path/model_path'

In [56]:
def loadimgs(path,n = 0):
    #path => Path of train directory or test directory
    X=[]
    y = []
    cat_dict = {}
    lang_dict = {}
    curr_y = n
    # we load every alphabet seperately so we can isolate them later
    for alphabet in os.listdir(path):
        print("loading alphabet: " + alphabet)
        lang_dict[alphabet] = [curr_y,None]
        alphabet_path = os.path.join(path,alphabet)
        # every letter/category has it's own column in the array, so  load seperately
        for letter in os.listdir(alphabet_path):
            cat_dict[curr_y] = (alphabet, letter)
            category_images=[]
            letter_path = os.path.join(os.path.sep, alphabet_path, letter)
            # read all the images in the current category
            for filename in os.listdir(letter_path):
                image_path = os.path.join(letter_path, filename)
                audio, sample_rate = torchaudio.load(image_path)
                mel_spectrogram = T.MelSpectrogram(
                    sample_rate=sample_rate,
                    n_fft=n_fft,
                    win_length=win_length,
                    hop_length=hop_length,
                    center=True,
                    pad_mode="reflect",
                    power=2.0,
                    norm="slaney",
                    onesided=True,
                    n_mels=n_mels,
                    mel_scale="htk",)
            
            
            

                melspec1 = mel_spectrogram(audio)
                #image = imread(image_path)
                category_images.append(melspec1)
                y.append(curr_y)
            try:
                X.append(np.stack(category_images))
            # edge case  - last one
            except ValueError as e:
                print(e)
                print("error - category_images:", category_images)
            curr_y += 1
            lang_dict[alphabet][1] = curr_y - 1
    y = np.vstack(y)
    X = np.stack(X)
    return X,y,lang_dict

In [57]:
def loadimgs(path,n = 0):
    n_fft = 1024
    win_length = None
    hop_length = 512
    n_mels = 128
    X=[]
    y = []
    cat_dict = {}
    lang_dict = {}
    curr_y = n
    
    # we load every alphabet seperately so we can isolate them later
    for alphabet in os.listdir(path):
        print("loading alphabet: " + alphabet)
        lang_dict[alphabet] = [curr_y,None]
        alphabet_path = os.path.join(path,alphabet)
        # every letter/category has it's own column in the array, so  load seperately
        for letter in os.listdir(alphabet_path):
            cat_dict[curr_y] = (alphabet, letter)
            category_images=[]
            letter_path = os.path.join(os.path.sep, alphabet_path, letter)
            
            audio, sample_rate = torchaudio.load(letter_path)
            mel_spectrogram = T.MelSpectrogram(
                sample_rate=sample_rate,
                n_fft=n_fft,
                win_length=win_length,
                hop_length=hop_length,
                center=True,
                pad_mode="reflect",
                power=2.0,
                norm="slaney",
                onesided=True,
                n_mels=n_mels,
                mel_scale="htk",)
        
            
            
            
            

            melspec1 = mel_spectrogram(audio)
                #image = imread(image_path)
            #category_images.append(melspec1)
            
            y.append(curr_y)
       
            X.append(np.stack(melspec1))
            #X.append(np.stack(category_images))
           
        curr_y += 1
        lang_dict[alphabet]= curr_y - 1
    y = np.vstack(y)
    X = np.stack(X)
    return X,y,lang_dict

In [59]:
#Xtrain,ytrain,train_classes=loadimgs(train_folder)

In [10]:
train_classes

{'allergies': 0, 'asthma': 1, 'covid': 2, 'other': 3, 'pneumonia': 4}

In [11]:
Xtrain.shape

(1407, 1, 128, 44)

In [12]:
#ytrain[1000:1100]

In [13]:
print(Xtrain.shape)
print(Xtrain[0,0,127])
print(ytrain.shape)
print(ytrain[0:41])
print(train_classes)

(1407, 1, 128, 44)
[3.4123819e-09 1.7818673e-09 8.1085810e-10 8.6461183e-10 1.6441156e-09
 7.9906548e-10 1.0252131e-09 1.4825569e-09 1.5650010e-09 1.1782888e-09
 1.1812680e-09 1.6109997e-09 1.6060866e-09 1.7248326e-09 9.2110303e-10
 1.0983032e-09 1.2673312e-09 6.6256628e-10 1.3618981e-09 9.1256958e-10
 9.3971730e-10 8.5434676e-10 1.8080178e-09 1.2740200e-09 1.2785099e-09
 2.3729785e-09 6.6838735e-10 1.1942630e-09 1.2515776e-09 9.1384195e-10
 1.1206148e-09 1.3866291e-09 1.3528293e-09 1.1620692e-09 9.4175756e-10
 1.2915077e-09 2.1587891e-09 1.1599064e-09 1.3061235e-09 1.3922936e-09
 1.5494037e-09 2.0490079e-09 1.7729729e-09 9.0257291e-10]
(1407, 1)
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
{'allergies': 0, 'asthma': 1, 'covid': 2, 'other': 3, 'pneumonia': 4}


In [14]:
Xtrain[1406].shape

(1, 128, 44)

In [15]:
Xval,yval,val_classes=loadimgs(val_folder)

loading alphabet: allergies
loading alphabet: asthma
loading alphabet: covid
loading alphabet: other
loading alphabet: pneumonia


In [16]:
print(Xtrain.shape)
print(Xval.shape)

(1407, 1, 128, 44)
(348, 1, 128, 44)


In [ ]:
#(1407, 1, 128, 44)train shape
# (348, 1, 128, 44)

In [17]:
def get_siamese_model(input_shape):
    #Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    seq = Sequential()
    seq.add(Conv2D(32, (3,3), activation='relu', input_shape=input_shape,data_format='channels_first',
                kernel_initializer=initializers.RandomNormal(stddev=1e-2), kernel_regularizer=l2(2e-4)))
    seq.add(MaxPooling2D())
    seq.add(Conv2D(64, (3,3), activation='relu',data_format='channels_first',
                kernel_initializer=initializers.RandomNormal(stddev=1e-2),
                bias_initializer=initializers.RandomNormal(mean=0.5,stddev=1e-2), kernel_regularizer=l2(2e-4)))
    seq.add(MaxPooling2D())
    seq.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-2),
                bias_initializer=initializers.RandomNormal(mean=0.5,stddev=1e-2), kernel_regularizer=l2(2e-4)))
    seq.add(MaxPooling2D())
    seq.add(Conv2D(256, (4,4), activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-2),
                bias_initializer=initializers.RandomNormal(mean=0.5,stddev=1e-2), kernel_regularizer=l2(2e-4)))
    seq.add(Flatten())
    seq.add(Dense(1407, activation='sigmoid',
                kernel_regularizer=l2(1e-3),
                kernel_initializer=initializers.RandomNormal(stddev=1e-2),
                bias_initializer=initializers.RandomNormal(mean=0.5,stddev=1e-2)))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = seq(left_input)
    encoded_r = seq(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    #L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    #L1_distance = L1_layer([encoded_l, encoded_r])
    L1_distance = K.abs(encoded_l-encoded_r)

    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',
                   bias_initializer=initializers.RandomNormal(mean=0.5,stddev=1e-2))(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net, seq

In [18]:
model, seq = get_siamese_model(( 1, 128,44))
model.summary()
seq.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 128, 44)  0           []                               
                                ]                                                                 
                                                                                                  
 input_2 (InputLayer)           [(None, 1, 128, 44)  0           []                               
                                ]                                                                 
                                                                                                  
 sequential (Sequential)        (None, 1407)         40238719    ['input_1[0][0]',                
                                                                  'input_2[0][0]']            

In [1]:
#from tensorflow.keras.utils import plot_model
#plot_model(model, to_file=os.path.join(save_path,"siamese_net.png"))
#plot_model(seq, to_file=os.path.join(save_path,"seq_net.png"))

In [19]:
optimizer = Adam(learning_rate = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)

In [32]:
def get_batch(batch_size,s="train"):
    """
    Create batch of n pairs, half same class, half different class
    """
    if s == 'train':
        X = Xtrain
        categories = train_classes
    else:
        X = Xval
        categories = val_classes
    n_classes, n_examples, w, h = X.shape
    
    # randomly sample several classes to use in the batch
    categories = rng.choice(n_classes,size=(batch_size,),replace=False)
    
    # initialize 2 empty arrays for the input image batch
    pairs=[np.zeros((batch_size, 1, w, h )) for i in range(2)]
    
    # initialize vector for the targets
    targets=np.zeros((batch_size,))
    
    # make one half of it '1's, so 2nd half of batch has same class
    targets[batch_size//2:] = 1
    for i in range(batch_size):
        category = categories[i]
        idx_1 = rng.randint(0, n_examples)
        pairs[0][i,:,:,:] = X[category, idx_1].reshape(1, w, h)
        idx_2 = rng.randint(0, n_examples)
        
        # pick images of same class for 1st half, different for 2nd
        if i >= batch_size // 2:
            category_2 = category  
        else: 
            # add a random number to the category modulo n classes to ensure 2nd image has a different category
            category_2 = (category + rng.randint(1,n_classes)) % n_classes
        
        pairs[1][i,:,:,:] = X[category_2,idx_2].reshape(1, w, h)
    
    return pairs, targets

In [33]:
pairs,targets=get_batch(4)
print(pairs[0].shape)
print(pairs[1].shape)
print(targets.shape)


(4, 1, 128, 44)
(4, 1, 128, 44)
(4,)


In [26]:
def generate(batch_size, s="train"):
    """a generator for batches, so model.fit_generator can be used. """
    while True:
        pairs, targets = get_batch(batch_size,s)
        yield (pairs, targets)

In [40]:
def make_oneshot_task(N, s="val", language=None):
    """Create pairs of test image, support set for testing N way one-shot learning. """
    if s == 'train':
        X = Xtrain
        categories = train_classes
    else:
        X = Xval
        categories = val_classes
    n_classes, n_examples, w, h = X.shape
    
    indices = rng.randint(0, n_examples,size=(N,))
    if language is not None: # if language is specified, select characters for that language
        low, high = categories[language]
        if N > high - low:
            raise ValueError("This language ({}) has less than {} letters".format(language, N))
        categories = rng.choice(range(low,high),size=(N,),replace=False)

    else: # if no language specified just pick a bunch of random letters
        categories = rng.choice(range(n_classes),size=(N,),replace=False)            
    true_category = categories[0]
    ex1, ex2 = rng.choice(n_examples,replace=True,size=(2,))
    test_image = np.asarray([X[true_category,ex1,:,:]]*N).reshape(N, 1, w , h)
    support_set = X[categories,indices,:,:]
    support_set[0,:,:] = X[true_category,ex2]
    support_set = support_set.reshape(N, 1, w, h )
    targets = np.zeros((N,))
    targets[0] = 1
    targets, test_image, support_set = shuffle(targets, test_image, support_set)
    pairs = [test_image,support_set]

    return pairs, targets

In [41]:
def test_oneshot(model, N, k, s = "val", verbose = 0):
    """Test average N way oneshot learning accuracy of a siamese neural net over k one-shot tasks"""
    n_correct = 0
    if verbose:
        print("Evaluating model on {} random {} way one-shot learning tasks ... \n".format(k,N))
    for i in range(k):
        inputs, targets = make_oneshot_task(N,s)
        probs = model.predict(inputs)
        if np.argmax(probs) == np.argmax(targets):
            n_correct+=1
    percent_correct = (100.0 * n_correct / k)
    if verbose:
        print("Got an average of {}% {} way one-shot learning accuracy \n".format(percent_correct,N))
    return percent_correct

In [52]:
# Hyper parameters
evaluate_every = 200 # interval for evaluating on one-shot tasks
batch_size = 32
n_iter = 10# No. of training iterations
N_way = 20 # how many classes for testing one-shot tasks
n_val = 250 # how many one-shot tasks to validate on
best = -1

In [53]:
tf.config.run_functions_eagerly(True)

In [54]:
print("Starting training process!")
print("-------------------------------------")
t_start = time.time()
for i in range(1, n_iter+1):
    (inputs,targets) = get_batch(batch_size)
    loss = model.train_on_batch(inputs, targets)
    if i % evaluate_every == 0:
        print("\n ------------- \n")
        print("Time for {0} iterations: {1} mins".format(i, (time.time()-t_start)/60.0))
        print("Train Loss: {0}".format(loss)) 
        val_acc = test_oneshot(model, N_way, n_val, verbose=True)
        model.save_weights(os.path.join(model_path, 'weights.{}.h5'.format(i)))
        if val_acc >= best:
            print("Current best: {0}, previous best: {1}".format(val_acc, best))
            best = val_acc

Starting training process!
-------------------------------------


In [48]:
def nearest_neighbour_correct(pairs,targets):
    """returns 1 if nearest neighbour gets the correct answer for a one-shot task
        given by (pairs, targets)"""
    L2_distances = np.zeros_like(targets)
    for i in range(len(targets)):
        L2_distances[i] = np.sum(np.sqrt(pairs[0][i]**2 - pairs[1][i]**2))
    if np.argmin(L2_distances) == np.argmax(targets):
        return 1
    return 0

In [49]:
def test_nn_accuracy(N_ways,n_trials):
    """Returns accuracy of NN approach """
    print("Evaluating nearest neighbour on {} unique {} way one-shot learning tasks ...".format(n_trials,N_ways))

    n_right = 0
    
    for i in range(n_trials):
        pairs,targets = make_oneshot_task(N_ways,"val")
        correct = nearest_neighbour_correct(pairs,targets)
        n_right += correct
    return 100.0 * n_right / n_trials

In [50]:
ways = np.arange(1,20,2)
resume =  False
trials = 50

In [51]:
val_accs, train_accs,nn_accs = [], [], []
for N in ways:    
    val_accs.append(test_oneshot(model, N, trials, "val", verbose=True))
    train_accs.append(test_oneshot(model, N, trials, "train", verbose=True))
    nn_acc = test_nn_accuracy(N, trials)
    nn_accs.append(nn_acc)
    print ("NN Accuracy = ", nn_acc)
    print("---------------------------------------------------------------------------------------------------------------")

Evaluating model on 50 random 1 way one-shot learning tasks ... 

1/1 [==============================] - 0s 32ms/step
Got an average of 100.0% 1 way one-shot learning accuracy 

Evaluating model on 50 random 1 way one-shot learning tasks ... 

1/1 [==============================] - 0s 32ms/step
Got an average of 100.0% 1 way one-shot learning accuracy 

Evaluating nearest neighbour on 50 unique 1 way one-shot learning tasks ...
NN Accuracy =  100.0
---------------------------------------------------------------------------------------------------------------
Evaluating model on 50 random 3 way one-shot learning tasks ... 

1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 30ms/step
Got an average of 100.0% 3 way one-shot learning accuracy 

Evaluating model on 50 random 3 way one-shot learning tasks ... 

1/1 [==============================] - 0s 31ms/step
Got an average of 100.0% 3 way one-shot learning accuracy 

Evaluating nearest neighbour on 50 unique 3 way one-shot learning tasks ...
NN Accuracy =  0.0
---------------------------------------------------------------------------------------------------------------
Evaluating model on 50 random 5 way one-shot learning tasks ... 



C:\Users\salta\AppData\Local\Temp/ipykernel_17232/2586585278.py:6: RuntimeWarning: invalid value encountered in sqrt
  L2_distances[i] = np.sum(np.sqrt(pairs[0][i]**2 - pairs[1][i]**2))


1/1 [==============================] - 0s 31ms/step
Got an average of 100.0% 5 way one-shot learning accuracy 

Evaluating model on 50 random 5 way one-shot learning tasks ... 

1/1 [==============================] - 0s 38ms/step
Got an average of 100.0% 5 way one-shot learning accuracy 

Evaluating nearest neighbour on 50 unique 5 way one-shot learning tasks ...
NN Accuracy =  0.0
---------------------------------------------------------------------------------------------------------------
Evaluating model on 50 random 7 way one-shot learning tasks ... 

1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 32ms/step
Got an average of 100.0% 7 way one-shot learning accuracy 

Evaluating model on 50 random 7 way one-shot learning tasks ... 

1/1 [==============================] - 0s 36ms/step
Got an average of 100.0% 7 way one-shot learning accuracy 

Evaluating nearest neighbour on 50 unique 7 way one-shot learning tasks ...
NN Accuracy =  0.0
---------------------------------------------------------------------------------------------------------------
Evaluating model on 50 random 9 way one-shot learning tasks ... 

1/1 [==============================] - 0s 34ms/step
Got an average of 100.0% 9 way one-shot learning accuracy 

Evaluating model on 50 random 9 way one-shot learning tasks ... 

1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 38ms/step
Got an average of 100.0% 9 way one-shot learning accuracy 

Evaluating nearest neighbour on 50 unique 9 way one-shot learning tasks ...
NN Accuracy =  0.0
---------------------------------------------------------------------------------------------------------------
Evaluating model on 50 random 11 way one-shot learning tasks ... 

1/1 [==============================] - 0s 34ms/step
Got an average of 100.0% 11 way one-shot learning accuracy 

Evaluating model on 50 random 11 way one-shot learning tasks ... 

1/1 [==============================] - 0s 35ms/step
Got an average of 100.0% 11 way one-shot learning accuracy 

Evaluating nearest neighbour on 50 unique 11 way one-shot learning tasks ...
NN Accuracy =  0.0
---------------------------------------------------------------------------------------------------------------
Evaluating model on 50 random 13 way one-shot learning tasks ... 

1/1 [==============================] - 0s 38ms

1/1 [==============================] - 0s 36ms/step
Got an average of 100.0% 13 way one-shot learning accuracy 

Evaluating model on 50 random 13 way one-shot learning tasks ... 

1/1 [==============================] - 0s 37ms/step
Got an average of 100.0% 13 way one-shot learning accuracy 

Evaluating nearest neighbour on 50 unique 13 way one-shot learning tasks ...
NN Accuracy =  0.0
---------------------------------------------------------------------------------------------------------------
Evaluating model on 50 random 15 way one-shot learning tasks ... 

1/1 [==============================] - 0s 36ms/step
Got an average of 100.0% 15 way one-shot learning accuracy 

Evaluating model on 50 random 15 way one-shot learning tasks ... 

1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 37ms/step
Got an average of 100.0% 15 way one-shot learning accuracy 

Evaluating nearest neighbour on 50 unique 15 way one-shot learning tasks ...
NN Accuracy =  0.0
---------------------------------------------------------------------------------------------------------------
Evaluating model on 50 random 17 way one-shot learning tasks ... 

1/1 [==============================] - 0s 38ms/step
Got an average of 100.0% 17 way one-shot learning accuracy 

Evaluating model on 50 random 17 way one-shot learning tasks ... 

1/1 [==============================] - 0s 36ms/step
Got an average of 100.0% 17 way one-shot learning accuracy 

Evaluating nearest neighbour on 50 unique 17 way one-shot learning tasks ...
NN Accuracy =  0.0
---------------------------------------------------------------------------------------------------------------
Evaluating model on 50 random 19 way one-shot learning tasks ... 

1/1 [==============================] - 0s 34

1/1 [==============================] - 0s 33ms/step
Got an average of 100.0% 19 way one-shot learning accuracy 

Evaluating model on 50 random 19 way one-shot learning tasks ... 

1/1 [==============================] - 0s 33ms/step
Got an average of 100.0% 19 way one-shot learning accuracy 

Evaluating nearest neighbour on 50 unique 19 way one-shot learning tasks ...
NN Accuracy =  0.0
---------------------------------------------------------------------------------------------------------------
